In [13]:
import cv2
import matplotlib.pyplot as plt

from deconvolution_processing import Process

from compare_methods import Compare

In [14]:
image_path = "og.png"
target_image_path = "target.png"

image = cv2.imread(image_path)
target_image = cv2.imread(target_image_path)

#normalizer = Normalization(target_image)
# result = normalizer.reinhard_normalization(image)
#
# plt.show()
# plt.imshow(result)
# plt.show()
# plt.imshow(target_image)
# plt.show()
# plt.imshow(target_image)
# plt.show()



In [15]:
image_path = "og.png"
target_image_path = "target.png"

image = cv2.imread(image_path)
target_image = cv2.imread(target_image_path)

#normalizer = Normalization(target_image)
#normalized = normalizer.reinhard_normalization(image)

deconv_processor = Process()
stain_images_wavelet, stain_matrix_wavelet, concentrations_wavelet = deconv_processor.process_image(
    image,
    visualise=True,
    method='wavelet'
)
stain_images_ruifork, stain_matrix_ruifork, concentrations_ruifork = deconv_processor.process_image(
    image,
    visualise=True,
    method='ruifork',
    threshold=0.15
)
stain_images_reference, stain_matrix_reference, concentrations_reference = deconv_processor.process_image(
    image,
    visualise=True,
    method='reference',
)
#stain_images_normalized = wavelet_processor.process_image(normalized, visualise=True)

ValueError: Nieznana metoda: wavelet